In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.optim as optim


from zenml.pipelines import pipeline
from zenml.steps import step, Output

import optuna
from optuna.trial import TrialState

In [2]:
@pipeline
def pytorch_hpo_pipeline(
    load_data,
    run_hpo,
    train_test,
):
    """A `pipeline` to load data, load model, and train/evaluate the model."""
    train_dataloader, test_dataloader = load_data()
    best_hparams = run_hpo(train_dataloader, test_dataloader)
    train_test(best_hparams, train_dataloader, test_dataloader)


In [3]:
def get_mnist():
    """A `step` to load the Fashion MNIST dataset as a tuple of torch Datasets."""
    batch_size = 64

    # Download training data from open datasets.
    training_data = datasets.FashionMNIST(
        root="data",
        train=True,
        download=True,
        transform=ToTensor(),
    )

    # Download test data from open datasets.
    test_data = datasets.FashionMNIST(
        root="data",
        train=False,
        download=True,
        transform=ToTensor(),
    )

    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=batch_size)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)

    return train_dataloader, test_dataloader

@step
def load_data() -> Output(
    train_dataloader=DataLoader, test_dataloader=DataLoader
):
    train_dataloader, test_dataloader = get_mnist()

    return train_dataloader, test_dataloader

In [4]:
def define_hpo_model(trial):
    CLASSES = 10
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []
    in_features = 28 * 28
    layers.append(nn.Flatten())
    
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))
        in_features = out_features
        
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)

def objective(trial, train_dataloader, test_dataloader):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Generate model
    model = define_hpo_model(trial)
    
    # Get dataset
    # train_dataloader, valid_dataloader = get_mnist()
    
    # Train model
    model = model.to(device)
    loss_fn = nn.CrossEntropyLoss()
    # lr=1e-3
    # optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    optimizer_name = trial.suggest_categorical("optimizer_name", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    
    size = len(train_dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    
    size = len(test_dataloader.dataset)
    num_batches = len(test_dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in test_dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    test_accuracy = 100*correct
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
    return test_accuracy


@step
def run_hpo(
    train_dataloader: DataLoader, 
    test_dataloader: DataLoader
) -> dict:
    
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(trial, train_dataloader, test_dataloader), n_trials=30, timeout=600)
    
    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    
    best = trial.params
    print("Found best hparam dict")
    print(best)
    
    return best

In [5]:
def define_model(hparam:dict):
    CLASSES = 10
    layers = []
    in_features = 28 * 28
    layers.append(nn.Flatten())
    
    print(hparam)
    
    for i in range(hparam['n_layers']):
        out_features = hparam[f"n_units_l{i}"]
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = hparam[f"dropout_l{i}"]
        layers.append(nn.Dropout(p))
        in_features = out_features
        
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)

In [6]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"

def train(dataloader, model, loss_fn, optimizer):
    """A function to train a model for one epoch."""
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    """A function to test a model on the validation / test dataset."""
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    test_accuracy = 100*correct
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    return test_accuracy

In [7]:
@step
def train_test(
    best_hparams: dict,
    train_dataloader: DataLoader, 
    test_dataloader: DataLoader
) -> Output(trained_model=nn.Module, test_acc=float):
    """A `step` to train and evaluate a torch model on given dataloaders."""
    
    epochs = 5
    
    model = define_model(best_hparams)
    
    print("Training optimized model:")
    print(model)

    model = model.to(device)
    loss_fn = nn.CrossEntropyLoss()
    
    lr = best_hparams['lr']
    optimizer_name = best_hparams['optimizer_name']
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    
    print("Using optimized hparams")
    print(optimizer_name)
    print(lr)
    
    # optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    test_acc = 0
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        test_acc = test(test_dataloader, model, loss_fn)
    print("Done!")

    return model, test_acc

In [8]:
pytorch_hpo_pipeline(
    load_data=load_data(),
    run_hpo=run_hpo(),
    train_test=train_test(),
).run(unlisted=True)

Unable to find ZenML repository in your current working directory (/home/dnth/Desktop/zenml_hpo) or any parent directories. If you want to use an existing repository which is in a different location, set the environment variable 'ZENML_REPOSITORY_PATH'. If you want to create a new repository, run zenml init.
Running without an active repository root.
Running unlisted pipeline on stack default (caching enabled)
Step load_data has started.
Step load_data has finished in 0.113s.
Step run_hpo has started.


[I 2022-11-28 13:59:39,073] A new study created in memory with name: no-name-27cc2b47-740e-4bdf-814d-bd85b13a2eda
[I 2022-11-28 13:59:45,261] Trial 0 finished with value: 56.61000000000001 and parameters: {'n_layers': 3, 'n_units_l0': 43, 'dropout_l0': 0.2815373416933857, 'n_units_l1': 81, 'dropout_l1': 0.26580914247201665, 'n_units_l2': 9, 'dropout_l2': 0.3804910152892979, 'optimizer_name': 'SGD', 'lr': 0.018576818121802653}. Best is trial 0 with value: 56.61000000000001.


Test Error: 
 Accuracy: 56.6%, Avg loss: 1.153342 



[I 2022-11-28 13:59:49,895] Trial 1 finished with value: 68.19 and parameters: {'n_layers': 3, 'n_units_l0': 34, 'dropout_l0': 0.4666109869394437, 'n_units_l1': 75, 'dropout_l1': 0.2040654298545448, 'n_units_l2': 30, 'dropout_l2': 0.495855491006952, 'optimizer_name': 'RMSprop', 'lr': 0.0004397464868680228}. Best is trial 1 with value: 68.19.


Test Error: 
 Accuracy: 68.2%, Avg loss: 0.802256 



...

Test Error: 
 Accuracy: 82.1%, Avg loss: 0.493544 



[I 2022-11-28 14:01:44,490] Trial 28 finished with value: 80.47999999999999 and parameters: {'n_layers': 2, 'n_units_l0': 113, 'dropout_l0': 0.2026168914676933, 'n_units_l1': 108, 'dropout_l1': 0.4389939683740725, 'optimizer_name': 'RMSprop', 'lr': 0.00022808188337547963}. Best is trial 26 with value: 83.31.


Test Error: 
 Accuracy: 80.5%, Avg loss: 0.543703 



[I 2022-11-28 14:01:48,751] Trial 29 finished with value: 79.55 and parameters: {'n_layers': 1, 'n_units_l0': 51, 'dropout_l0': 0.2876720747177252, 'optimizer_name': 'Adam', 'lr': 0.017565461501013218}. Best is trial 26 with value: 83.31.


Test Error: 
 Accuracy: 79.5%, Avg loss: 0.572266 

Study statistics: 
  Number of finished trials:  30
  Number of pruned trials:  0
  Number of complete trials:  30
Best trial:
  Value:  83.31
  Params: 
    n_layers: 1
    n_units_l0: 118
    dropout_l0: 0.23036559984831573
    optimizer_name: RMSprop
    lr: 0.00137201635481431
Found best hparam dict
{'n_layers': 1, 'n_units_l0': 118, 'dropout_l0': 0.23036559984831573, 'optimizer_name': 'RMSprop', 'lr': 0.00137201635481431}
Step run_hpo has finished in 2m9s.
Step train_test has started.
{'n_layers': 1, 'n_units_l0': 118, 'dropout_l0': 0.23036559984831573, 'optimizer_name': 'RMSprop', 'lr': 0.00137201635481431}
Training optimized model:
Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=118, bias=True)
  (2): ReLU()
  (3): Dropout(p=0.23036559984831573, inplace=False)
  (4): Linear(in_features=118, out_features=10, bias=True)
  (5): LogSoftmax(dim=1)
)
Using optimized hparams
RMSprop
0.00